In [1]:
%load_ext autoreload
%autoreload 2
import torch
import sys
sys.path.append('../scripts/')
from dataset import get_loader, split_datasets, pre_process_data
import pandas as pd
import glob
from loss import calculate_bce
from train import train_script
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
from train import train_script
from model import VGG16
import torchvision.models as models
import torch.optim as optim
from tqdm import tqdm
from metrics import calculate_precision, calculate_recall, calculate_cf
from PIL import Image
import collections

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
train, val, test = split_datasets(pre_process_data('../../dog_cats_data/train/'), 0.1)

In [4]:
print(f'Train set: {len(train)}')
print(f'Val set: {len(val)}')
print(f'Test set: {len(test)}')

Train set: 20250
Val set: 2250
Test set: 2500


In [5]:
train_loader = get_loader(data = train, img_size = 16, batch_size = 512, num_workers=6, is_train = True, shuffle = True)
val_loader = get_loader(data = val, img_size = 16, batch_size = 512, shuffle = True, is_train = False, num_workers=6)

In [10]:
# train_script(VGG16(), train_loader, val_loader, 50, device, note = 'Double batch norm and double dropout image size 16 with in main conv block. Optim changed to SGD.')

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)
